In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from datetime import datetime, timedelta,date
import plotly.express as px
import requests
import io

%matplotlib inline

COVID-19: Spread Progression

In [67]:
url = "http://www.pref.kanagawa.jp/osirase/1369/data/csv/patient.csv"
r = requests.get(url)
df_patient= pd.read_csv(io.BytesIO(r.content),sep=",",encoding="shift-jis")
df_patient.tail()

,発表日,居住地,年代,性別
680,2020-04-17,神奈川県厚木保健福祉事務所管内,20代,男性
681,2020-04-17,神奈川県厚木保健福祉事務所管内,60代,女性
682,2020-04-17,神奈川県厚木保健福祉事務所管内,30代,男性
683,2020-04-17,神奈川県藤沢市,60代,女性
684,2020-04-17,神奈川県藤沢市,50代,男性


In [46]:
#参照する位置情報は事前ダウンロードする。直接CSVリンクがない。
df_lat_long_file = pd.read_csv("kanagawa-14_2018.csv",encoding = "shift-jis")
df_lat_long_file.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,14,神奈川県,14101,横浜市鶴見区,141010001001,本町通一丁目,35.502643,139.680165,0,3
1,14,神奈川県,14101,横浜市鶴見区,141010002001,潮田町一丁目,35.504476,139.682396,0,3
2,14,神奈川県,14101,横浜市鶴見区,141010003000,寛政町,35.501344,139.701084,0,1
3,14,神奈川県,14101,横浜市鶴見区,141010002004,潮田町四丁目,35.503095,139.692805,0,3
4,14,神奈川県,14101,横浜市鶴見区,141010004000,東寺尾東台,35.503422,139.663808,0,1


In [68]:
df_patient['居住地'].unique()

array(['神奈川県', '神奈川県相模原市', '神奈川県横浜市', '神奈川県鎌倉保健福祉事務所管内',
       '神奈川県厚木保健福祉事務所管内', '神奈川県小田原保健福祉事務所管内', '神奈川県平塚保健福祉事務所管内',
       '神奈川県藤沢市', '神奈川県川崎市', '神奈川県茅ヶ崎市保健所管内', '神奈川県横須賀市', 'スペイン（横浜市発表）',
       '国外（川崎市発表）', '川崎市外（川崎市発表）', '神奈川県川崎市外（川崎市発表）', '神奈川県茅ケ崎市保健所管内及び都内',
       '神奈川県茅ケ崎市保健所管内', '横浜市内', '横須賀市内', '川崎市内', '相模原市内', '藤沢市内', '神奈川県内',
       '川崎市外', '横浜市外', '東京都\u3000'], dtype=object)

In [69]:
#df_data["発表日"] = pd.to_datetime( df_data["発表日"]).dt.strftime('%m/%d/%Y')

"市外","（〇〇市発表）"の居住地は削除して、居住地検索するときに影響しないようにする。

In [71]:
df_data = df_patient[~df_patient['居住地'].str.contains("外")]
df_data = df_data[~df_data['居住地'].str.contains('\（')]
df_data

,発表日,居住地,年代,性別
0,2020-01-16,神奈川県,30代,男性
1,2020-02-11,神奈川県,50代,男性
2,2020-02-13,神奈川県,80代,女性
3,2020-02-14,神奈川県,30代,男性
4,2020-02-17,神奈川県相模原市,40代,女性
...,...,...,...,...
680,2020-04-17,神奈川県厚木保健福祉事務所管内,20代,男性
681,2020-04-17,神奈川県厚木保健福祉事務所管内,60代,女性
682,2020-04-17,神奈川県厚木保健福祉事務所管内,30代,男性
683,2020-04-17,神奈川県藤沢市,60代,女性


In [72]:
df_data['居住地'].unique()

array(['神奈川県', '神奈川県相模原市', '神奈川県横浜市', '神奈川県鎌倉保健福祉事務所管内',
       '神奈川県厚木保健福祉事務所管内', '神奈川県小田原保健福祉事務所管内', '神奈川県平塚保健福祉事務所管内',
       '神奈川県藤沢市', '神奈川県川崎市', '神奈川県茅ヶ崎市保健所管内', '神奈川県横須賀市',
       '神奈川県茅ケ崎市保健所管内及び都内', '神奈川県茅ケ崎市保健所管内', '横浜市内', '横須賀市内', '川崎市内',
       '相模原市内', '藤沢市内', '神奈川県内', '東京都\u3000'], dtype=object)

In [51]:
df_lat_log = df_lat_long_file[["市区町村名","緯度","経度"]]
#市レベルまでの精度でよいので一つずつ代表だけ抽出
df_lat_log.drop_duplicates(subset = "市区町村名",inplace = True)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
cut_after_shi = lambda s: s.split("市")[0]
cut_after_gun = lambda s: s.split("郡")[0]

In [53]:
df_lat_log.loc[:,"市区町村名"] = df_lat_log["市区町村名"].apply(cut_after_shi)
df_lat_log.loc[:,"市区町村名"] = df_lat_log["市区町村名"].apply(cut_after_gun)
df_lat_log

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,市区町村名,緯度,経度
0,横浜,35.502643,139.680165
130,横浜,35.469008,139.621716
257,横浜,35.467075,139.619367
325,横浜,35.422175,139.673702
583,横浜,35.438080,139.621009
736,横浜,35.461988,139.571635
798,横浜,35.420463,139.620282
874,横浜,35.337428,139.622072
975,横浜,35.522765,139.617895
1076,横浜,35.386415,139.502309


In [54]:
#Dictionary型にして、Map表示の準備をする。
dict_lat_log = dict(zip(df_lat_log['市区町村名'] , zip(df_lat_log['緯度'],df_lat_log['経度'])))
dict_lat_log

{'横浜': (35.550709000000005, 139.59042),
 '川崎': (35.585701, 139.497123),
 '相模原': (35.50144, 139.404535),
 '横須賀': (35.206801, 139.61791599999998),
 '平塚': (35.320696000000005, 139.370693),
 '鎌倉': (35.320847, 139.497922),
 '藤沢': (35.335188, 139.49716899999999),
 '小田原': (35.248714, 139.12328),
 '茅ヶ崎': (35.328485, 139.375121),
 '逗子': (35.281544, 139.610675),
 '三浦': (35.258634, 139.593622),
 '秦野': (35.361692, 139.24338999999998),
 '厚木': (35.415655, 139.35238600000002),
 '大和': (35.507309, 139.4337),
 '伊勢原': (35.427170000000004, 139.24606799999998),
 '海老名': (35.449496, 139.382276),
 '座間': (35.501027, 139.414615),
 '南足柄': (35.29364, 139.13307),
 '綾瀬': (35.420321, 139.442378),
 '高座': (35.365845, 139.37102),
 '中': (35.295390999999995, 139.24855),
 '足柄上': (35.336532, 139.11971499999999),
 '足柄下': (35.171964, 139.10195900000002),
 '愛甲': (35.484274, 139.262291)}

In [73]:
df_data['居住地'] = df_data['居住地'].str.replace('茅ケ崎','茅ヶ崎')

In [74]:
for index in dict_lat_log.keys() :
    change_add = lambda s: index if index in s else s 
#    change_add = lambda s: index if True else s 
    df_data['居住地'] = df_data['居住地'].apply(change_add)
#    df_data.loc[df_data['居住地'].str.contains(index),"Lat"] = dict_lat_log[index][0]
#    df_data.loc[df_data['居住地'].str.contains(index),"Long"] = dict_lat_log[index][1]

In [75]:
df_data= df_data.groupby(['居住地']).count()


In [80]:
df_data = df_data.reset_index()
df_data

,居住地,発表日,年代,性別,Lat,Long
0,厚木,55,55,55,,
1,小田原,18,18,18,,
2,川崎,145,145,145,,
3,平塚,41,41,41,,
4,東京都,1,1,1,,
5,横浜,215,129,129,,
6,横須賀,32,32,32,,
7,相模原,55,55,55,,
8,神奈川県,4,4,4,,
9,神奈川県内,1,1,1,,


In [81]:
df_data["Lat"] = ""
df_data["Long"] = ""
for index in dict_lat_log.keys() :
    df_data.loc[df_data['居住地'].str.contains(index),"Lat"] = dict_lat_log[index][0]
    df_data.loc[df_data['居住地'].str.contains(index),"Long"] = dict_lat_log[index][1]

In [82]:
#辞書から市名が見つからなかったのはそもそも表示ができないので対象にしない
df_data = df_data[~(df_data["Lat"] == "")]
df_data = df_data.reset_index(drop = True)
df_data

,居住地,発表日,年代,性別,Lat,Long
0,厚木,55,55,55,35.4157,139.352
1,小田原,18,18,18,35.2487,139.123
2,川崎,145,145,145,35.5857,139.497
3,平塚,41,41,41,35.3207,139.371
4,横浜,215,129,129,35.5507,139.59
5,横須賀,32,32,32,35.2068,139.618
6,相模原,55,55,55,35.5014,139.405
7,茅ヶ崎,19,19,19,35.3285,139.375
8,藤沢,41,41,41,35.3352,139.497
9,鎌倉,46,46,46,35.3208,139.498


Todo: 累積データにした方がよい

In [86]:
kanagawa = folium.Map(location=[35.550709000000005, 139.59042], zoom_start=9,max_zoom=12,min_zoom=4,height=500,width="80%")
for i in range(0,len(df_data.index)):
    folium.Circle(
        location=[df_data.iloc[i]['Lat'], df_data.iloc[i]['Long']],

        radius=(int(np.log2(df_data.iloc[i]['年代']+1)))*1000,
        color='#ff6600',
        fill_color='#ff8533',
        fill=True).add_to(kanagawa)

kanagawa